In [1]:
!pip install --quiet \
    "sciencebeam-parser>=0.1.4" \
    "tensorflow<2.0.0" \
    "numpy<1.17.0" \
    "pandas<1.3.0" \
    "typing_extensions"

In [2]:
!python --version

Python 3.7.6


In [3]:
!pip freeze | grep --ignore-case --extended-regexp "tensorflow|keras|sciencebeam|numpy|pandas|delft"

delft==0.2.7
Keras==2.2.4
Keras-Applications==1.0.8
keras-bert==0.84.0
keras-embed-sim==0.9.0
keras-layer-normalization==0.15.0
keras-multi-head==0.28.0
keras-pos-embd==0.12.0
keras-position-wise-feed-forward==0.7.0
Keras-Preprocessing==1.1.2
keras-self-attention==0.50.0
keras-transformer==0.39.0
numpy==1.16.6
pandas==1.2.5
sciencebeam-parser==0.1.4
sciencebeam-trainer-delft==0.0.31
tensorflow==1.15.5
tensorflow-estimator==1.15.1


In [4]:
import logging
import os
from pathlib import Path

import tensorflow as tf

In [5]:
# configure logging so that we see ScienceBeam Parser's output
logging.basicConfig(level='ERROR')

# reduce tensorflow warnings
tf.logging.set_verbosity(tf.logging.ERROR)

In [6]:
# get a sample document
sample_url = 'https://github.com/elifesciences/sciencebeam-parser/raw/develop/test-data/minimal-example.pdf'
local_sample_file = tf.keras.utils.get_file(os.path.basename(sample_url), sample_url)

In [7]:
# create an instance of ScienceBeamParser
from sciencebeam_parser.resources.default_config import DEFAULT_CONFIG_FILE
from sciencebeam_parser.config.config import AppConfig
from sciencebeam_parser.utils.media_types import MediaTypes
from sciencebeam_parser.app.parser import ScienceBeamParser


config = AppConfig.load_yaml(DEFAULT_CONFIG_FILE)

# the parser contains all of the models
sciencebeam_parser = ScienceBeamParser.from_config(config)

Using TensorFlow backend.


In [8]:
# preload models
sciencebeam_parser.fulltext_models.preload()

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator FunctionTransformer from version 0.20.4 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.20.4 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator Pipeline from version 0.20.4 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [9]:
# a session provides a scope and temporary directory for intermediate files
# it is recommended to create a separate session for every document
with sciencebeam_parser.get_new_session() as session:
    session_source = session.get_source(
        local_sample_file,
        MediaTypes.PDF
    )
    converted_file = session_source.get_local_file_for_response_media_type(
        MediaTypes.TEI_XML
    )
    # Note: the converted file will be in the temporary directory of the session
    print(Path(converted_file).read_text(encoding='utf-8'))

<TEI xmlns="http://www.tei-c.org/ns/1.0"><teiHeader><fileDesc><titleStmt><title level="a" type="main" coords="1,107.60,84.88,312.10,13.28">This minimal example is used as a test document for automated tests</title></titleStmt></fileDesc><note type="O" coords="1,56.80,57.28,117.06,40.88"><hi rend="bold">Title:</hi> Minimal Example <hi rend="bold">Abstract:</hi></note><note type="O" coords="1,435.89,84.88,4.05,13.28">.</note></teiHeader></TEI>
